In [ ]:
import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer
import matplotlib.pyplot as plt

# Read the DataFrame from an Excel file
excel_file_path = 'Sample_Imp.xlsx'  # Replace with your actual file path
df = pd.read_excel(excel_file_path)

# Display the DataFrame before imputation
print("DataFrame before imputation:")
print(df)

# Replace '#VALUE!' with NaN for proper handling
df.replace('#VALUE!', np.nan, inplace=True)

# Convert the columns to numeric (remove commas and convert to float)
df.iloc[:, 1:] = df.iloc[:, 1:].replace('[\$,]', '', regex=True).astype(float)

# Convert the first column to datetime
df['Month End'] = pd.to_datetime(df['Month End'])

# Set the 'Month End' column as the index
df.set_index('Month End', inplace=True)

# Identify columns with missing values
columns_with_missing = df.columns[df.isna().any()].tolist()

# Initialize the IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=42)

# Perform imputation only for columns with missing values
df_imputed = df.copy()
df_imputed[columns_with_missing] = imputer.fit_transform(df[columns_with_missing])

# Display the DataFrame after imputation
print("\nDataFrame after imputation:")
print(df_imputed)

# Combine original and imputed data into a single DataFrame
df_combined = pd.concat([df, df_imputed.add_suffix('_imputed')], axis=1)

# Save the combined DataFrame to an Excel file with XlsxWriter engine
with pd.ExcelWriter('output_data_combined.xlsx', engine='xlsxwriter') as writer:
    df_combined.to_excel(writer, sheet_name='Combined Data', index=True)

    # Create a worksheet for the plot
    worksheet = writer.sheets['Combined Data']

    # Create a chart object
    chart = writer.book.add_chart({'type': 'line'})

    # Configure the series for the chart
    for column in columns_with_missing:
        chart.add_series({
            'name': f'Original {column}',
            'categories': ['Combined Data', 1, 0, len(df_combined), 0],
            'values': ['Combined Data', 1, df_combined.columns.get_loc(column) + 1, len(df_combined), df_combined.columns.get_loc(column) + 1],
            'line': {'dash_type': 'dash'},
            'marker': {'type': 'o', 'size': 6},
        })
        chart.add_series({
            'name': f'Imputed {column}',
            'categories': ['Combined Data', 1, 0, len(df_combined), 0],
            'values': ['Combined Data', 1, df_combined.columns.get_loc(f'{column}_imputed') + 1,
                       len(df_combined), df_combined.columns.get_loc(f'{column}_imputed') + 1],
            'line': {'dash_type': 'solid'},
            'marker': {'type': 'x', 'size': 8},
        })

    # Set chart title and axis labels
    chart.set_title({'name': 'Comparison of Original and Imputed Columns'})
    chart.set_x_axis({'name': 'Month End'})
    chart.set_y_axis({'name': 'Value'})

    # Insert the chart into the worksheet
    worksheet.insert_chart('H2', chart)

print("Excel file with combined data and graph has been created.")

